In [ ]:
from openai import OpenAI

import pandas as pd
import json
import os

# INITIALIZE GPT 4
api_key = "" 
org_key = "" 

apikey_path = "../apikeys/api_openai_llamaindex.key"
orgKey_path = "../apikeys/api_openai_org.key"

with open(apikey_path, 'r') as file:
    api_key = file.read().strip()

with open(orgKey_path, 'r') as file:
    org_key = file.read().strip()
    
openai_client = OpenAI(api_key=api_key, organization=org_key) 

In [ ]:

# gpt-3.5-turbo-1106
# gpt-4-1106-preview

def GPT(system_prompt, user_prompt):
    result = "" 
    prompt_tokens = 0 
    completion_tokens = 0
    total_tokens = 0

    try:
        response = openai_client.chat.completions.create(
            messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}],
            temperature=0,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            max_tokens=1500,
            model="gpt-4-1106-preview",
        )     
        #result =  response["choices"][0]["message"]["content"].strip()
        result = response.choices[0].message.content 
        prompt_tokens = response.usage.prompt_tokens
        completion_tokens = response.usage.completion_tokens
        total_tokens = response.usage.total_tokens

        return result, prompt_tokens, completion_tokens, total_tokens
    
    except Exception as e:
        print(f"An error occurred on GPT: {e}")
        return None, 0, 0, 0
    
system_prompt = "You are a very helpful assistant. You will help me summarize a meeting transcript." 

#### Read the transcript

In [ ]:
transcript_path = 'meeting3_transcription.txt'

In [ ]:
# Open the file and read its content into a variable
with open(transcript_path, 'r', encoding='utf-8') as file:
    transcript = file.read()

#### Summary prompt

In [ ]:
summary_prompt = f"""Meeting:
{transcript}

Objective: Generate increasingly concise, entity-dense summaries of the meeting and action items.

Process: Repeat the following steps five times:

Step 1: Identify 1-3 informative entities (separated by semicolons) missing from the previously generated summary.
Step 2: Write a new, denser summary of the same length that covers all entities and details from the previous summary, including the missing entities.

Missing Entities Criteria:
Relevant: Pertinent to the main discussion.
Specific: Descriptive yet concise (5 words or fewer).
Novel: Not present in the previous summary.
Faithful: Reflects information discussed during the meeting.
Anywhere: Can appear anywhere in the new summary.

Guidelines:
The initial summary should be long (4-5 sentences, ~80 words) but highly nonspecific, using verbose language and fillers (e.g., “the meeting covered”) to reach the word count.

Optimize each word: Rewrite the previous summary to improve flow and create space for additional entities.

Condense by fusing, compressing, and removing uninformative phrases (e.g., “the discussion focused on”).

The summaries should become highly dense and self-contained, easily understood without referring to the meeting notes.

Missing entities can be inserted anywhere in the new summary.

Never drop entities from the previous summary; if space is limited, add fewer new entities.

By the end provide entities with sentence description as Meeting summary.
"""

summary, prompt_tokens, completion_tokens, total_tokens = GPT(system_prompt, summary_prompt)

In [ ]:
print(summary) 

In [ ]:

tasks_prompt = f"""From the following TRANSCRIPT can you please create a detailed list of tasks assigned to participants.  Please make sure to start each task with the person assigned to it.

TRANSCRIPT:
{transcript}

TASKS:
""" 

tasks, prompt_tokens, completion_tokens, total_tokens = GPT(system_prompt, tasks_prompt)

In [ ]:
print(tasks)

In [ ]:

priorities_prompt = f"""From the following TRANSCRIPT please identify the most prioritary to-do actions and who is assigned to each one of them. Please assign a priority tag like HIGH, MEDIUM, LOW and order the list according to these priorities.

In the list please format it like so: <priority> - <person in charge>: <description> 

For example:

HIGH - Cindy: Connect with GTM teams to ensure support for corporate events.

TRANSCRIPT:
{transcript}

PRIORITIES:
""" 

priorities, prompt_tokens, completion_tokens, total_tokens = GPT(system_prompt, priorities_prompt)

In [ ]:
print(priorities)